# Steps
1. Corpus Loading and Cleaning
- mention number of data points
1. Import packages
1. Preprocessing pipeline
- load the dataset
- list what all you did on it
- Dataset analysis
1. Extract Features (Tokens)
1. Train / Fine tune model on it
- Set training parameters (What all are you fine tuning)
- Save model configurations
1. Input question, parse and extract features
1. Score Metric : Rouge and BLEU
1. Post processing
- Finding the outliers
1. Model Inference

1. Filters Outputs (Dependency Parsing)

In [ ]:
# !pip install -U transformers
# !pip install -U datasets
# !pip install tensorboard
# !pip install sentencepiece
# !pip install accelerate
# !pip install evaluate
# !pip install rouge

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=cceabde775be42895e9e7acc46b25fe572182364c1c56e2e65a836a6148a52bc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


- T5 : Unit Text-to-Text encoder-decoder model (Using T5ForConditionalGeneration)

- Evaluate : Used to evaluate the huggingface models for different tasks
  1. Rogue or BLEU Score

In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, RandomSampler

import pprint
import evaluate

import transformers
from transformers import (
    T5Tokenizer,
    T5Model,
    T5TokenizerFast,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)

from datasets import load_dataset

import json
from tqdm import tqdm


import nltk
import spacy
import string

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

- Fine Tuning T5 model
- Batch size and number of processors is 4
- Max context length for input is 256 and target is 32
- Inputs below this will be padded and above will be truncated

In [ ]:
# Setup Model parameters and stuff

TOKENIZER = T5TokenizerFast.from_pretrained("t5-base")

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

MODEL = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
MODEL.to(DEVICE)

OPTIMIZER = Adam(MODEL.parameters(), lr=0.00001)
Q_LEN = 256   # Question Length
T_LEN = 32    # Target Length
BATCH_SIZE = 4
# DEVICE = "cuda:0"


[Squad Dataset](https://rajpurkar.github.io/SQuAD-explorer/)

In [ ]:
from pathlib import Path

path = Path("/content/drive/MyDrive/Shared With Others/NLP Model Lab/squad_train.json")

In [ ]:
with open(path) as f:
  data = json.load(f)

In [ ]:
f

<_io.TextIOWrapper name='/content/drive/MyDrive/Shared With Others/NLP Model Lab/squad_train.json' mode='r' encoding='UTF-8'>

prepare_data : Takes in the data, extracts the contenxt and returns the articles list containing extracted questions and answers

In [ ]:
def prepare_data(data):
    articles = []

    for article in data["data"]:
        for paragraph in article["paragraphs"]:
            for qa in paragraph["qas"]:
                question = qa["question"]

                if not qa["is_impossible"]:
                  answer = qa["answers"][0]["text"]

                inputs = {"context": paragraph["context"], "question": question, "answer": answer}

                articles.append(inputs)

    return articles

In [ ]:
data = prepare_data(data)

In [ ]:
data = pd.DataFrame(data)

In [ ]:
print(data.head(5))

                                             context  \
0  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
1  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
2  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
3  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
4  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   

                                            question               answer  
0           When did Beyonce start becoming popular?    in the late 1990s  
1  What areas did Beyonce compete in when she was...  singing and dancing  
2  When did Beyonce leave Destiny's Child and bec...                 2003  
3      In what city and state did Beyonce  grow up?        Houston, Texas  
4         In which decade did Beyonce become famous?           late 1990s  


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130319 entries, 0 to 130318
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   context   130319 non-null  object
 1   question  130319 non-null  object
 2   answer    130319 non-null  object
dtypes: object(3)
memory usage: 3.0+ MB


- Dataset has 1.3L rows and 3 cols

In [ ]:
data.shape

(130319, 3)

- Creating Custom Dataset (Pytorch)

- T5 Tokenizer is used here

In [ ]:
# tokenizer = T5Tokenizer.from_pretrained(MODEL)

# # Target Tokenizers
# with tokenizer.as_target_tokenizer():
#     labels = tokenizer(
#         targets,
#         max_length=MAX_LENGTH,
#         truncation=True,
#         padding='max_length'
#     )

In [ ]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]

        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length",
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)

        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100

        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }

- Using Dataloader to load and Preprocess the dataset

In [ ]:
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(TOKENIZER, data, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)

## Training and Validation

- In training, input_ids, attention_mask, labels, and decoder_attention_mask are passed into model

- T5 Base consists of approx 223 million parameters

In [ ]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(1):
  MODEL.train()
  for batch in tqdm(train_loader, desc="Training batches"):
    input_ids = batch["input_ids"].to(DEVICE)
    attention_mask = batch["attention_mask"].to(DEVICE)
    labels = batch["labels"].to(DEVICE)
    decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

    # print(input_ids.device)
    # print(attention_mask.device)
    # print(labels.device)
    # print(decoder_attention_mask.device)

    # print(MODEL.device)

    outputs = MODEL(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels,
        decoder_attention_mask=decoder_attention_mask
        )

    OPTIMIZER.zero_grad()
    outputs.loss.backward()
    OPTIMIZER.step()
    train_loss += outputs.loss.item()
    train_batch_count += 1
    if train_batch_count >= len(train_loader) * 0.01:
      break



  #Evaluation
  MODEL.eval()

  for batch in tqdm(val_loader, desc="Validation batches"):
    input_ids = batch["input_ids"].to(DEVICE)
    attention_mask = batch["attention_mask"].to(DEVICE)
    labels = batch["labels"].to(DEVICE)
    decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

    outputs = MODEL(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels,
        decoder_attention_mask=decoder_attention_mask
        )

    OPTIMIZER.zero_grad()
    outputs.loss.backward()
    OPTIMIZER.step()
    val_loss += outputs.loss.item()
    val_batch_count += 1

    print(f"{epoch+1}/{2} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")

Validation batches:   0%|          | 1/6516 [00:00<32:26,  3.35it/s]

1/2 -> Train loss: 0.9319736608143511	Validation loss: 1.3941826820373535


Validation batches:   0%|          | 2/6516 [00:00<32:16,  3.36it/s]

1/2 -> Train loss: 0.9319736608143511	Validation loss: 0.80363579839468


Validation batches:   0%|          | 3/6516 [00:00<32:25,  3.35it/s]

...

1/2 -> Train loss: 0.9319736608143511	Validation loss: 0.5067395170645015


Validation batches: 100%|█████████▉| 6515/6516 [33:41<00:00,  3.23it/s]

1/2 -> Train loss: 0.9319736608143511	Validation loss: 0.5066685133988891


Validation batches: 100%|██████████| 6516/6516 [33:42<00:00,  3.22it/s]

1/2 -> Train loss: 0.9319736608143511	Validation loss: 0.5067118016097917


In [ ]:
# Save the results

MODEL.save_pretrained("qa_model")
TOKENIZER.save_pretrained("qa_tokenizer")

('qa_tokenizer/tokenizer_config.json',
 'qa_tokenizer/special_tokens_map.json',
 'qa_tokenizer/spiece.model',
 'qa_tokenizer/added_tokens.json',
 'qa_tokenizer/tokenizer.json')

## Evaluation : BLUE and Rouge
- Computed using n-gram similarity

- ROUGE1: It is the ratio of the number of words that match the predictions and ground truth to the number of words in the predictions.
- ROUGE2: It is the ratio of the number bi-grams that match in the predictions and the ground truth to the number of bi-grams in the predictions.

In [ ]:
rouge = evaluate.load("rouge")

In [ ]:
def predict_answer(context, question, ref_answer=None):
    inputs = TOKENIZER(question, context, max_length=Q_LEN, padding="max_length", truncation=True, add_special_tokens=True)

    input_ids = torch.tensor(inputs["input_ids"], dtype=torch.long).to(DEVICE).unsqueeze(0)
    attention_mask = torch.tensor(inputs["attention_mask"], dtype=torch.long).to(DEVICE).unsqueeze(0)

    outputs = MODEL.generate(input_ids=input_ids, attention_mask=attention_mask)

    predicted_answer = TOKENIZER.decode(outputs.flatten(), skip_special_tokens=True)

    if ref_answer:
        # Load the Bleu metric
        bleu = evaluate.load("google_bleu")
        score = bleu.compute(predictions=[predicted_answer],
                            references=[ref_answer])

        print("Context: \n", context)
        print("\n")
        print("Question: \n", question)
        return {
            "Reference Answer: ": ref_answer,
            "Predicted Answer: ": predicted_answer,
            "BLEU Score: ": score
        }
    else:
        return predicted_answer

In [ ]:
context = data.iloc[0]['context']
question = data.iloc[0]['question']
answer = data.iloc[0]['answer']


predict_answer(context, question, answer)

Context: 
 Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".


Question: 
 When did Beyonce start becoming popular?


{'Reference Answer: ': 'in the late 1990s',
 'Predicted Answer: ': 'late 1990s',
 'BLEU Score: ': {'google_bleu': 0.3}}

In [ ]:
context = "He is the longest-serving prime minister from outside the Indian National Congress. Modi was born and raised in Vadnagar in northeastern Gujarat, where he completed his secondary education. He was introduced to the RSS at the age of eight."
question = 'Where was modi raised'

predict_answer(context, question)

'Vadnagar'

In [ ]:
!pip install locale

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
!pip install Faker

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
import faker

fake = faker.Faker()

paragraph = fake.paragraph()
print(paragraph)

question = fake.question(question_mark=True)
print(question)


ModuleNotFoundError: No module named 'faker'